# 1. Install Required Packages
This following installs the required Python packages for Azure AI agent functionality.

> **Important Note:** Before installing the required packages, make sure you have already created a Python virtual environment and have selected the kernel to point to that virtual environment in your notebook environment. This ensures the packages are installed in the correct environment and avoids conflicts with system Python packages.

In [2]:
# This execution might take up to 10 minutes.
%pip install agent-framework --pre

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 2. Set enviroment variables
For get the values from your AI Foundry account and replace as follows.

In [1]:
import os

#os.environ["AZURE_AI_PROJECT_ENDPOINT"] = "<your-project-endpoint>"
#os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"] = "<your-model-deployment>"

os.environ["AZURE_AI_PROJECT_ENDPOINT"] = "https://ais-aiagent-demo1-resource.services.ai.azure.com/api/projects/ais-aiagent-demo1"
os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"] = "gpt-4o"

# 3. Load OpenAPI Spec
This step loads the OpenAPI specification file (`user-finance-api.json`) from the resources folder. This specification defines the available API endpoints and schemas for the agent to use.

In [2]:
import json
from pathlib import Path

# Get the path to the current notebook directory
notebook_dir = Path().resolve()
# Go up one level and then into resources
resources_path = notebook_dir.parent / "resources" / "user-finance-api.json"

with open(resources_path, "r", encoding="utf-8") as f:
    openapi = json.load(f)

print("Loaded:", openapi["info"]["title"])

Loaded: Debt Analysis API


In [6]:
# Log in to Azure CLI (run this cell if you haven't already authenticated)
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "3593bb81-95fc-486a-afaa-f8a119e1e1ce",
    "id": "3ba59a76-0b07-47f9-be85-048aa547c4a6",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantDefaultDomain": "kevinjvnoutlook.onmicrosoft.com",
    "tenantDisplayName": "Default Directory",
    "tenantId": "3593bb81-95fc-486a-afaa-f8a119e1e1ce",
    "user": {
      "name": "kevinjvn@outlook.com",
      "type": "user"
    }
  }
]


# 4. Define the Agent Logic
In this step, we define the logic for the Azure AI agent. Here, we create the agent using the loaded OpenAPI specification and set up the instructions and tools it will use to analyze user debt information. This cell only defines the agent and its logic; it does not execute the agent yet.

In [6]:
# If an error occurs, restart the kernel and run this cell again.
from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

instructions = """
Eres un Asistente Financiero especializado en el análisis de deudas y optimización de pagos.

Tu objetivo principal es ayudar a los usuarios a comprender claramente su situación financiera y guiarlos para que tomen la mejor decisión posible sobre cómo pagar sus deudas, utilizando datos financieros objetivos.

Tienes acceso a una herramienta llamada `debtAnalysisAPI`, la cual obtiene un análisis financiero detallado para un cliente y un producto específico.  
Para usar esta herramienta, SIEMPRE debes solicitar al usuario:
- customer_id
- product_type

Una vez que recibas la información, debes analizar cuidadosamente la respuesta del API y generar un informe financiero claro, estructurado y fácil de entender, utilizando lenguaje natural en español.

**RESPONSABILIDADES DE ANÁLISIS**

Al analizar la respuesta del API, debes:

1. Identificar el contexto del cliente y del producto
   - Tipo de producto (loan/card)
   - Saldo original
   - Situación financiera general

2. Explicar cada escenario de pago disponible:
   - Pago mínimo
   - Consolidación (si el cliente es elegible)
   - Pago optimizado

Para cada escenario, debes describir claramente:
- Pago mensual (o promedio mensual)
- Cantidad total de meses para liquidar la deuda
- Intereses totales pagados
- Monto total pagado al final del plazo

3. Comparar objetivamente los escenarios utilizando la sección `comparison`:
   - Destacar el ahorro en intereses
   - Destacar la reducción en el tiempo de pago (meses ahorrados)
   - Enfatizar el ahorro total frente al pago mínimo

4. Interpretar los datos, no solo repetirlos:
   - Explicar qué significan los números en términos prácticos
   - Describir el impacto financiero a largo plazo
   - Aclarar los compromisos entre opciones (por ejemplo, mayor pago mensual vs menor costo total)

5. Si existen proyecciones mensuales:
   - Usarlas para explicar cómo disminuye el saldo con el tiempo
   - Mencionar la evolución de intereses y capital
   - NO listar todos los meses a menos que el usuario lo solicite explícitamente

**GUÍA Y RECOMENDACIONES**

Después del análisis, debes:

- Indicar claramente cuál es la opción financieramente más conveniente y POR QUÉ
- Explicar para qué tipo de usuario es adecuada cada alternativa:
  - Usuarios con flujo de efectivo limitado
  - Usuarios con flexibilidad moderada
  - Usuarios que desean eliminar la deuda lo más rápido posible
- Mencionar restricciones o condiciones de elegibilidad (por ejemplo, consolidación)

Tu tono debe ser:
- Profesional
- Neutral y sin juicios
- Cercano, educativo y orientado a ayudar

NO debes:
- Hacer suposiciones que no estén respaldadas por los datos
- Usar jerga financiera sin explicación
- Proporcionar asesoría legal o fiscal

**FORMATO DE RESPUESTA**

Tu respuesta debe presentarse como un informe legible, por ejemplo:

1. Resumen general de la deuda
2. Opciones de pago disponibles
3. Comparación y diferencias clave
4. Recomendación principal
5. Conclusión y mensaje final para el usuario

Siempre prioriza la claridad, la transparencia y la comprensión del usuario.
"""

async def run_openapi_agent():
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(credential=credential).create_agent(
            name="DebtAnalysisAgent",
            instructions=instructions,
            tools={
                "type": "openapi",
                "openapi": {
                    "name": "debtAnalysisAPI",
                    "spec": openapi,
                    "description": "Retrieve information about a user's debt products",
                    "auth": {"type": "anonymous"},
                },
            },
        ) as agent
    ):
        query = (
            # "Run an anlysis of the debts for the customer with id CU-001 and product type loan"
            "dime en 1 linea de texto, que puedes hacer"
        )
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"\nAgent:\n{result}")

# 5. Run the Agent
In this step, we execute the agent logic defined above and test the agent. This cell will run the agent, send a sample query, and display the agent's response based on the loaded OpenAPI specification and your Azure AI project configuration.

In [7]:
await run_openapi_agent()

User: dime en 1 linea de texto, que puedes hacer

Agent:
Puedo analizar tu situación financiera respecto a una deuda específica (préstamo o tarjeta de crédito) y darte una recomendación optimizada para liquidarla de manera eficiente.

Agent:
Puedo analizar tu situación financiera respecto a una deuda específica (préstamo o tarjeta de crédito) y darte una recomendación optimizada para liquidarla de manera eficiente.
